In [1]:
import pandas as pd
import numpy as np
import pycountry
import countrydict

In [2]:
df = pd.read_excel("emdat_public_2023_02_14_query_uid-AJtVn9.xlsx",skiprows = 6)

In [3]:
df = df.join(pd.DataFrame(np.zeros(len(df))))
df = df.rename(columns = {'0':'Region'})

In [4]:
for i in range(len(df)):
    df['Region'][i] = countrydict.countries_to_regions[df['ISO'][i]]

C:\Users\noahd\AppData\Local\Temp\ipykernel_11144\2990388048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Region'][i] = countrydict.countries_to_regions[df['ISO'][i]]


In [5]:
df['Region']

0                    WAF
1                    SAS
2                    SCA
3                    SCA
4                    SCA
              ...       
16540                SEA
16541               ESAF
16542                 NZ
16543    {ENA, CNA, WNA}
16544                WCE
Name: Region, Length: 16545, dtype: object